## Recurrent Neural Network
Predict daily high and low temperatures?

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf

# workaround for MacOS/jupyter notebook bug w/ tensorflow
# https://www.programmersought.com/article/69923598438/
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [19]:
# https://www.tensorflow.org/tutorials/load_data/numpy
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
weather_pd = pd.read_csv('../data/weather.csv', index_col = 0)
weather_pd = weather_pd.drop(['DAY', 'STP', 'GUST'], axis=1)

In [20]:
weather_pd.head()

,YEAR,MONTH,SEASON,TEMP,DEWP,SLP,VISIB,WDSP,MXSPD,MAX,MIN,PRCP,SNDP
0,2000,1,0,47.6,38.1,1023.7,8.3,3.0,10.1,66.9,33.1,0.00,0.0
1,2000,1,0,55.3,46.3,1024.2,9.5,4.8,14.0,70.0,33.1,0.00,0.0
2,2000,1,0,62.6,55.4,1021.3,8.4,8.5,14.0,73.9,43.0,0.00,0.0
3,2000,1,0,65.2,58.6,1014.4,9.5,15.3,28.0,73.9,55.0,0.00,0.0
4,2000,1,0,45.7,30.9,1019.8,9.8,6.4,11.1,57.9,37.0,0.34,0.0


In [21]:
weather_pd.dtypes

YEAR        int64
MONTH       int64
SEASON      int64
TEMP      float64
DEWP      float64
SLP       float64
VISIB     float64
WDSP      float64
MXSPD     float64
MAX       float64
MIN       float64
PRCP      float64
SNDP      float64
dtype: object

In [22]:
# next day's weather
target = weather_pd[['MAX', 'MIN']].iloc[1:, :]
target.head()

,MAX,MIN
1,70.0,33.1
2,73.9,43.0
3,73.9,55.0
4,57.9,37.0
5,51.1,25.0


In [23]:
# today's weather
feature = weather_pd.iloc[:-1, :] # don't have next day on last day
feature.head()

,YEAR,MONTH,SEASON,TEMP,DEWP,SLP,VISIB,WDSP,MXSPD,MAX,MIN,PRCP,SNDP
0,2000,1,0,47.6,38.1,1023.7,8.3,3.0,10.1,66.9,33.1,0.00,0.0
1,2000,1,0,55.3,46.3,1024.2,9.5,4.8,14.0,70.0,33.1,0.00,0.0
2,2000,1,0,62.6,55.4,1021.3,8.4,8.5,14.0,73.9,43.0,0.00,0.0
3,2000,1,0,65.2,58.6,1014.4,9.5,15.3,28.0,73.9,55.0,0.00,0.0
4,2000,1,0,45.7,30.9,1019.8,9.8,6.4,11.1,57.9,37.0,0.34,0.0


In [24]:
weather = tf.data.Dataset.from_tensor_slices((feature.values, target.values))

In [25]:
for feat, targ in weather.take(5):
    print ('Features: {}, Target: {}'.format(feat, targ))

Features: [2.0000e+03 1.0000e+00 0.0000e+00 4.7600e+01 3.8100e+01 1.0237e+03
 8.3000e+00 3.0000e+00 1.0100e+01 6.6900e+01 3.3100e+01 0.0000e+00
 0.0000e+00], Target: [70.  33.1]
Features: [2.0000e+03 1.0000e+00 0.0000e+00 5.5300e+01 4.6300e+01 1.0242e+03
 9.5000e+00 4.8000e+00 1.4000e+01 7.0000e+01 3.3100e+01 0.0000e+00
 0.0000e+00], Target: [73.9 43. ]
Features: [2.0000e+03 1.0000e+00 0.0000e+00 6.2600e+01 5.5400e+01 1.0213e+03
 8.4000e+00 8.5000e+00 1.4000e+01 7.3900e+01 4.3000e+01 0.0000e+00
 0.0000e+00], Target: [73.9 55. ]
Features: [2.0000e+03 1.0000e+00 0.0000e+00 6.5200e+01 5.8600e+01 1.0144e+03
 9.5000e+00 1.5300e+01 2.8000e+01 7.3900e+01 5.5000e+01 0.0000e+00
 0.0000e+00], Target: [57.9 37. ]
Features: [2.0000e+03 1.0000e+00 0.0000e+00 4.5700e+01 3.0900e+01 1.0198e+03
 9.8000e+00 6.4000e+00 1.1100e+01 5.7900e+01 3.7000e+01 3.4000e-01
 0.0000e+00], Target: [51.1 25. ]


In [43]:
# batches of days - ues previous 3 days to predict 4th day?
days_in_batches = 4
batches = weather.batch(days_in_batches, drop_remainder=True)

In [44]:
# see batches of days
for input_ex, target_ex in batches.take(5):
    print('Input data: ', input_ex)
    print('Target data: ', target_ex)

Input data:  tf.Tensor(
[[2.0000e+03 1.0000e+00 0.0000e+00 4.7600e+01 3.8100e+01 1.0237e+03
  8.3000e+00 3.0000e+00 1.0100e+01 6.6900e+01 3.3100e+01 0.0000e+00
  0.0000e+00]
 [2.0000e+03 1.0000e+00 0.0000e+00 5.5300e+01 4.6300e+01 1.0242e+03
  9.5000e+00 4.8000e+00 1.4000e+01 7.0000e+01 3.3100e+01 0.0000e+00
  0.0000e+00]
 [2.0000e+03 1.0000e+00 0.0000e+00 6.2600e+01 5.5400e+01 1.0213e+03
  8.4000e+00 8.5000e+00 1.4000e+01 7.3900e+01 4.3000e+01 0.0000e+00
  0.0000e+00]
 [2.0000e+03 1.0000e+00 0.0000e+00 6.5200e+01 5.8600e+01 1.0144e+03
  9.5000e+00 1.5300e+01 2.8000e+01 7.3900e+01 5.5000e+01 0.0000e+00
  0.0000e+00]], shape=(4, 13), dtype=float64)
Target data:  tf.Tensor(
[[70.  33.1]
 [73.9 43. ]
 [73.9 55. ]
 [57.9 37. ]], shape=(4, 2), dtype=float64)
Input data:  tf.Tensor(
[[2.0000e+03 1.0000e+00 0.0000e+00 4.5700e+01 3.0900e+01 1.0198e+03
  9.8000e+00 6.4000e+00 1.1100e+01 5.7900e+01 3.7000e+01 3.4000e-01
  0.0000e+00]
 [2.0000e+03 1.0000e+00 0.0000e+00 3.6100e+01 2.7200e+01 1.032

In [45]:
# see expected output for each input
for i, (input_idx, target_idx) in enumerate(zip(input_ex[:5], target_ex[:5])):
    print("Step {:4d}".format(i))
    print("  input: {}".format(input_idx))
    print("  expected output: {}".format(target_idx))

Step    0
  input: [2.000e+03 1.000e+00 0.000e+00 4.300e+01 1.550e+01 1.027e+03 9.900e+00
 6.800e+00 1.500e+01 6.300e+01 3.200e+01 0.000e+00 0.000e+00]
  expected output: [33.1 21.2]
Step    1
  input: [2.0000e+03 1.0000e+00 0.0000e+00 2.6600e+01 1.4000e+01 1.0235e+03
 5.1000e+00 3.6000e+00 8.0000e+00 3.3100e+01 2.1200e+01 9.0000e-02
 3.1000e+00]
  expected output: [39.9 21.9]
Step    2
  input: [2.0000e+03 1.0000e+00 0.0000e+00 3.0500e+01 2.7200e+01 1.0168e+03
 5.4000e+00 2.7000e+00 8.0000e+00 3.9900e+01 2.1900e+01 1.0000e-01
 2.0000e+00]
  expected output: [45.  24.1]
Step    3
  input: [2.0000e+03 1.0000e+00 0.0000e+00 3.5700e+01 2.8400e+01 1.0061e+03
 6.3000e+00 7.1000e+00 1.5000e+01 4.5000e+01 2.4100e+01 2.6000e-01
 2.0000e+00]
  expected output: [45. 21.]


In [46]:
# make batches of the 4-day batches
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

batches = batches.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

batches

<BatchDataset shapes: ((64, 4, 13), (64, 4, 2)), types: (tf.float64, tf.float64)>

## Build the Model

In [76]:
# how many possible outputs
num_output_vals = 2 # find good number for this

# Number of RNN units
rnn_units = 64

In [77]:
def build_model(rnn_units, num_output_vals, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, batch_input_shape=[batch_size, 4, 13]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(num_output_vals)
    ])
    return model

In [78]:
model = build_model(rnn_units, num_output_vals, BATCH_SIZE)

## Try the Model

In [79]:
for input_example_batch, target_example_batch in batches.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, days_in_batches, num_output_vals)")

(64, 4, 2) # (batch_size, days_in_batches, num_output_vals)


In [80]:
target_example_batch.shape # 512 total # need another dim??

TensorShape([64, 4, 2])

In [81]:
input_example_batch.shape # 64*52=3328

TensorShape([64, 4, 13])

In [83]:
example_batch_predictions.shape

TensorShape([64, 4, 2])

In [84]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (64, 4, 128)              1792      
_________________________________________________________________
gru_8 (GRU)                  (64, 4, 64)               37248     
_________________________________________________________________
dense_17 (Dense)             (64, 4, 2)                130       
Total params: 39,170
Trainable params: 39,170
Non-trainable params: 0
_________________________________________________________________


In [85]:
# example predictions, all terrible bc model not trained
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([0, 0, 0, 0])

In [86]:
# show input and output
print("Input: \n", input_example_batch[0])
print()
print("Next Day Predictions: \n", sampled_indices)

Input: 
 tf.Tensor(
[[2.0030e+03 4.0000e+00 1.0000e+00 4.5100e+01 4.0900e+01 1.0237e+03
  6.7000e+00 9.1000e+00 1.4000e+01 5.9000e+01 3.9200e+01 3.9000e-01
  0.0000e+00]
 [2.0030e+03 4.0000e+00 1.0000e+00 4.2500e+01 4.1700e+01 1.0235e+03
  7.9000e+00 6.5000e+00 8.0000e+00 4.6900e+01 3.9900e+01 6.9000e-01
  0.0000e+00]
 [2.0030e+03 4.0000e+00 1.0000e+00 4.2900e+01 4.2200e+01 1.0166e+03
  6.9000e+00 8.2000e+00 1.2000e+01 4.5000e+01 3.9900e+01 1.2000e+00
  0.0000e+00]
 [2.0030e+03 4.0000e+00 1.0000e+00 4.4200e+01 4.3300e+01 1.0086e+03
  5.7000e+00 1.4100e+01 2.2900e+01 4.8000e+01 4.1000e+01 8.1000e-01
  0.0000e+00]], shape=(4, 13), dtype=float64)

Next Day Predictions: 
 [0 0 0 0]


In [87]:
# do we need logits for loss function? from_logits=True
loss = tf.losses.SparseCategoricalCrossentropy()
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, days_in_batches, num_output_vals)")
print("Mean loss:        ", mean_loss)
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

ValueError: Shape mismatch: The shape of labels (received (512,)) should equal the shape of logits except for the last dimension (received (256, 2)).

#### Resources Consulted
https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
